# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [1]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
embeddings_filename = "pushup_all"

pushup_both_sides_embeddings=emb.generate_embeddings(base_folder,embeddings_filename)

/home/romain.meunier@Digital-Grenoble.local/anaconda3/envs/ACV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1743670742.179471   24265 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743670742.271997   24320 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743670742.355657   24315 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743670742.427857   24313 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743670742.45

Images processed : 29
Images skipped : 0
CSV pathfile generated :./embeddings/pushup_all_embeddings.csv


In [21]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

base_folder = "./training_datasets/push-ups_images" 
embeddings_filename = "pushup_2a"
features=[
    'left_shoulder_elbow_wrist_angle',
    'right_shoulder_elbow_wrist_angle',
    ]

pushup_both_sides_embeddings=emb.generate_embeddings(base_folder,embeddings_filename,features,show_images=False)

I0000 00:00:1743672481.882870   26112 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743672481.972648   26240 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
W0000 00:00:1743672482.042719   26232 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743672482.083756   26233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Images processed : 29
Images skipped : 0
CSV pathfile generated :./embeddings/pushup_2a_embeddings.csv


In [ ]:
from ACV_Classifier_Fonctions import Embeddings

emb=Embeddings()

kaggle_path = "tr1gg3rtrash/yoga-posture-dataset"
embeddings_filename = "yoga_all"

pushup_both_sides_embeddings=emb.generate_embeddings_from_kaggle(kaggle_path,embeddings_filename)


---
# Classification

---

In [3]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_all_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

In [22]:
from ACV_Classifier_Fonctions import Classifier
from sklearn.tree import DecisionTreeClassifier

clf=Classifier()

model=DecisionTreeClassifier()
embeddings_filename='pushup_2a_embeddings.csv'

clf.fit_and_save_model(model,embeddings_filename)

DecisionTreeClassifier()

---
# Prediction

---

In [23]:
from ACV_Classifier_Fonctions import Predict

model_filename="pushup_2a_DecisionTreeClassifier.pkl"

model=Predict(model_filename)


I0000 00:00:1743672512.841916   26112 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743672512.889558   26272 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2


W0000 00:00:1743672512.959114   26265 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743672512.997755   26265 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [60]:
image_path='./training_datasets/push-ups_images/position_up/image_004.jpg'
model.predict(image_path)

['position_up', 'position_up', 'position_up']


'position_up'